In [1]:
import torch
from data_utils import GuidedGridMLMDataset, GuidedGridMLM_collate_fn
from torch.utils.data import DataLoader
from GridMLM_tokenizers import GuidedGridMLMTokenizer
from models import GuidedMLMH
from tqdm import tqdm
import numpy as np
from generate_utils import structured_progressive_generate, \
    random_progressive_generate, load_model, overlay_generated_harmony, \
    save_harmonized_score, generate_files_with_base2
import os

/home/maximos/miniconda3/envs/torch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# we need the folders of the actual musicXMLs, not the dataset,
# we need to load them through the tokenizer because we need the melody part
val_dir = '/media/maindisk/data/hooktheory_hr/hooktheory_all12_test'
jazz_dir = '/media/maindisk/data/gjt_melodies/gjt'

In [3]:
val_files = []
for dirpath, _, filenames in os.walk(val_dir):
    for file in filenames:
        if file.endswith('.xml') or file.endswith('.mxl') or file.endswith('.musicxml'):
            full_path = os.path.join(dirpath, file)
            val_files.append(full_path)
print('total data_files:', len(val_files))

jazz_files = []
for dirpath, _, filenames in os.walk(jazz_dir):
    for file in filenames:
        if file.endswith('.xml') or file.endswith('.mxl') or file.endswith('.musicxml'):
            full_path = os.path.join(dirpath, file)
            jazz_files.append(full_path)
print('total data_files:', len(jazz_files))

total data_files: 1886
total data_files: 650


In [4]:
tokenizer = GuidedGridMLMTokenizer(fixed_length=256)

In [5]:
device_name = 'cuda:2'
subfolder = 'all12'
curriculum_type='base2'
ablation = 'all'
model = load_model(curriculum_type = curriculum_type, subfolder=subfolder, ablation=ablation, \
        device_name = device_name, tokenizer=tokenizer)

In [6]:
jazz_idx = 0
val_idx = 0

guide_f = jazz_files[jazz_idx]
input_f = val_files[val_idx]

mxl_folder = 'examples_musicXML/' + subfolder + '/' + curriculum_type + '/' + ablation + '/'
midi_folder = 'examples_MIDI/' + subfolder + '/' + curriculum_type + '/' + ablation + '/'
os.makedirs(mxl_folder, exist_ok=True)
os.makedirs(midi_folder, exist_ok=True)

gen_harm, real_harm, guide_harm = generate_files_with_base2(
    model=model,
    tokenizer=tokenizer,
    input_f=input_f,
    guide_f=guide_f,
    mxl_folder=mxl_folder,
    midi_folder=midi_folder,
    name_suffix=0
)

/home/maximos/miniconda3/envs/torch/lib/python3.12/site-packages/music21/stream/base.py:3694: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)
/home/maximos/miniconda3/envs/torch/lib/python3.12/site-packages/music21/base.py:559: UserWarning: Setting an ID that could be mistaken for a memory location is discouraged: got 138042516821760
  warnings.warn(msg)
/home/maximos/miniconda3/envs/torch/lib/python3.12/site-packages/music21/stream/base.py:3694: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)
libpng warning: iCCP: known incorrect sRGB profile
Error at line 30 col 47: no instrument found for part 'P5f5740f56a79f0167da998a9b4cb4db2'
Error at line 988 col 47: no instrument found for part 'P184b6af55427159ae6cababfa67c9b45'
convert <examples_musicXML/all12/base2/all/gen_0.mxl>...
	to <examples_MIDI/all12/base2/all/gen_0.mid>
..